In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import random
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier


In [440]:
def _retData(file):
    dout = open('review_data.pkl', 'rb')
    data = pickle.load(dout)
    dout.close()
    trainDF = pd.DataFrame()
    trainDF['text'] = data['reviewText']
    trainDF['labels'] = data['overall']
    data_sample = trainDF.sample(frac=0.06, replace=False)
    return data_sample

In [455]:
def _vectorizers(which, df, what, Cvec):
    
    if what == 'train':
        if which=='cv':
            
            cvec_train = Cvec.transform(df['text'])
            penkiDF = pd.DataFrame(cvec_train.toarray())
            penkiDF['labels'] = df['labels'].values
            return penkiDF

        if which=='tf':
            
            train_tf = tfidf_vec.transform(df['text'])
            penkiDF_tf = pd.DataFrame(train_tf.toarray())
            penkiDF_tf['labels'] = df['labels'].values
            return penkiDF_tf
        
    if what == 'test':
        if which=='cv':
            
            cvec_test = Cvec.transform(df['test'])
            testDF = pd.DataFrame(cvec_test.toarray())
            return testDF
        
        if which=='tf':
            
            test_tf = tfidf_vec.transform(df['test'])
            test_df = pd.DataFrame(test_tf.toarray())
            return test_df

In [456]:
def _labeling(df):
    
    lis = df['labels'].values
    final = np.zeros(len(lis))
    
    for i in range(len(lis)):
        if(lis[i] == 1 or lis[i] == 2):
            final[i] = 1
        if(lis[i] == 3):
            final[i] = 2
        if(lis[i] == 4 or lis[i] == 5):
            final[i] = 3
    
    df['labels'] = pd.DataFrame(final).values
    return df

In [457]:
def _createModel(data, which):
    
    df1 = _labeling(data)
    if which =='cv':
        Cvec = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=n_feat).fit(data['text'])
        df = _vectorizers(which, df1, 'train', Cvec )
        
    if which == 'tf':
        tfidf_vec = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=n_feat).fit(data['text'])
        df = _vectorizers(which, df1, 'train', tfidf_vec )
    

    return df
    

In [463]:
def _knn(n_neigh, n_feat, dataframe):

    knn = KNeighborsClassifier(n_neighbors=n_neigh).fit(dataframe.iloc[:,0:n_feat:1],dataframe['labels'])
    filename = 'knn.pkl'
    pickle.dump(knn, open(filename, 'wb'))
    return knn

In [480]:
def predict_knn(model, text, which):
    
    rev = pd.DataFrame(text, columns=['test'])
    if which == 'cv':
        vecs = _vectorizers(which, rev, 'test', Cvec )
    if which == 'tf':
        vecs = _vectorizers(which, rev, 'test', tfidf_vec )
        
    return model.predict(vecs)


In [483]:
def _start():
    filename = 'review_data.pkl'
    data_sample = _retData(filename)
    dataframe = _createModel(data_sample, 'cv')
    return dataframe




In [495]:
if __name__ == '__main__':
    _strat()
    n_feat = 1000
    n_neigh = 3
    knn = _knn(n_neigh, n_feat, dataframe)
    which = 'cv'
    text = ['This bat cracked after two months\
        of light use since our league this \
        year still allowed the mako bats. Not worth the money and this isn’t the first one of these I \
        have seen crack this season.','I like the product but I discovered that there must be a lot of settling of the powder \
        because the canister appears to be about half full when first opened. . \
        I did not get the listed number of servings per canister so I weighed how much the provided green scoop delivers and \
        discovered that it provided 1/3 more product by weight than what was labeled on the "serving size". So just fill the \
        scoop 2/3 full or use a smaller scoop....that is what I did. I wrote the company but they did not seem to think \
        it was a problem. I did. When you use the correct amount it mixes and tastes much better.']
    print(predict_knn(knn, text, which))

[2. 3.]
